# Part 1
#### Note: Might use many resources, as I calc everything new for each cell / task to ensure partial usability.

### 3.1.a.a What is the probability to really see a name if C says so?

1.)in Newspaper: $P(N=Name|C=Yes) = \frac{P(C=yes|N=Name) * P(N=Name}{P(C=yes)} = \frac{0.9 * 0.05}{0.9*0.05 + 0.2*0.95} = 0.192$

2)in S.T: $P(N=Name|C=Yes) = \frac{P(C=yes|N=Name) * P(N=Name}{P(C=yes)} = \frac{0.9 * 0.01}{0.9*0.01 + 0.2*0.99} = 0.044$

### 3.1.a.b) How low must the false positive rate P(C=yes|N=not-name) get so that this probability goes up to 50% for both kinds of text?


Replace the false positivity rate (0.2) by x and the result by 0.5 and solve both equations of a) for x:
$$\frac{0.9 * 0.05}{0.9*0.05 + x*0.95} = 0.5, x = 0,047$$ 
$$\frac{0.9 * 0.01}{0.9*0.01 + x*0.99} = 0.5, x=0.009$$  
The false positivity rates have to be much lower in both cases (0.047 and 0.009) to achieve the desired probability

### 3.1.b are X and Y as defined in the following table independently distributed?  
 
X and Y are not independently distributed:
$$P(X_0)=0.4, P(X_1)=0.6, P(Y_a)=0.5, P(Y_b)=0.5$$  
$$P(X_0) * P(Y_a) = 0.4*0.5=0.2!=0.3 =P(X_0, Y_a)$$


### 3.1.c Compute log entropies for:

$H(X)=- \sum_{x \in X}{p(x) - log_2(p(x))}$  
a)  
$$H(X) = -0.4 * log(0.4) - 0.6 * log(0.6) = 0.971$$
$$H(Y) = -0.5*log(0.5) - 0.5 * log(0.5) = 1.000$$  
b)  
$$H(X, Y) = -0.3*log(0.3) - 0.1*log(0.1)- 0.2*log(0.2)-0.4*log(0.4) = 1.846$$  
$$H(X|Y) = H(X,Y) - H(Y) = 1.846 - 1.000 = 0.846$$
$$H(Y|X) = H(X,Y) - H(X) = 1.846 - 0.971 = 0.875 $$  





c)  
$$D(X||Y) = \sum_{x\in X}{p(x) * log (\frac{p(x)}{q(x)} )} = 0.4 * log(\frac{0.4}{0.5}) + 0.6 * log(\frac{0.6}{0.5}) = 0.029$$

# Part2

Download the homework data from Moodle. In the archive, you will find two files: Two German tokenized
text with 50K lines each. Each line consists of a sentence; special tokens have been added at the beginning
and at the end.


Example: %^% %^% Leder : Vielleicht ringt Normann nur um Anerkennung . %$% %$%

This sentence has 9 tokens, 10 bigrams and 11 trigrams: note that the special tokens %^% and %$% are
only considered if needed. Tokens are separated by a space character

In [1]:
# read the files
import os
from random import randint
prep = True
data_dir = './de_text'
train_file = 'de_text.test'
test_file = 'de_text.train'
trainfile = os.path.join(data_dir, train_file)
testfile = os.path.join(data_dir, test_file)
def preprocess(to_process, start_pattern, end_pattern):
    t = to_process
    if start_pattern:
        t = t.strip(start_pattern)
    if end_pattern:
        t = t.strip(end_pattern)
    return t

def read_and_prep(path_to_file, prep = True):
    with open(path_to_file, encoding = 'utf-8') as f:
        corp = [x.strip('\n') for x in f]
    if prep:
        corp = [preprocess(x,'%^% %^%', '%$% %$%') for x in corp]
    return corp

def get_sets():
    return read_and_prep(trainfile), read_and_prep(testfile)

train, test = get_sets()
print(len(train))
print(train[randint(0, len(train)-1)])
print(train[randint(0, len(train)-1)])

50000
Trotzdem ist die Aufgabe machbar " , sagte Hertha-Coach Falko Götz .
Zu akuten zerebralen Ereignissen kam es bei 2 Patienten , was im Rahmen der bisherigen Studien liegt .


### a) List the 20 most frequent words from the training set.

In [2]:
from operator import itemgetter

train, test = get_sets()

def get_n_grams(n, corpus):
    ngrams = {}
    for doc in corpus:
        #get token
        tokens = doc.split(" ")
        # pad it
        for i in range(n-1):
            tokens.insert(0, '<bos>')
            tokens.append('<eos>')
            
        # take n grams, starting at positon n-1, as it is the first non-pad token
        for i in range(n-1, len(tokens)):
            n_gram = tokens[i]
            
            # build n_grams by appending n-1 token from the left
            for j in range(1, n):
                n_gram = tokens[i-j] + " " + n_gram
            
            try:
                ngrams[n_gram] +=1
            except KeyError:
                ngrams[n_gram] = 1
    
    return ngrams

def get_n_most_freq(vocabulary, n=10):
    return sorted(vocabulary.items(), key = itemgetter(1), reverse=True)[:n]
    
train_vocab = get_n_grams(1, train)
n = 20
most_f = get_n_most_freq(train_vocab, n)
print(f"Top {n} most frequent token are {str(most_f)}")

Top 20 most frequent token are [('.', 51601), (',', 43454), ('der', 24425), ('die', 23464), ('und', 15684), ('in', 13147), ('"', 12934), ('den', 9621), ('von', 7450), ('zu', 7014), ('das', 6669), ('mit', 6434), ('sich', 6050), ('ist', 5955), ('auf', 5726), ('für', 5572), ('nicht', 5542), ('im', 5526), ('Die', 5429), ('des', 5186)]


### b) Compute the percentage of tokens in the test data that have not been seen in the training data

In [3]:
train, test = get_sets()

test_vocab = get_n_grams(1, test)

def get_unique_keys(dict1, dict2):
    unique = 0
    for k in dict1.keys():
        if k not in dict2:
            unique += 1
    return unique

unique_keys_in_test = get_unique_keys(test_vocab, train_vocab)
print(f"There are {len(test_vocab)} unique token in the test corpus. {unique_keys_in_test} of them are unique or {round(100 * unique_keys_in_test/len(test_vocab), 2)}% considering the vocabulary size")

There are 107126 unique token in the test corpus. 61854 of them are unique or 57.74% considering the vocabulary size


Counting all token in the corpus, the counts differ a lot:   

Number of unigrams in test-data: 907335  
Number token in test, but not in training: 67300

Percentage not seen in train: 0.07417326566262736

### c) List the 20 most frequent bigrams from the training set.

In [4]:
train, test = get_sets()
n = 20

train_bigrams = get_n_grams(2, train)
test_bigrams = get_n_grams(2, test)

most_freq_train_bi = get_n_most_freq(train_bigrams, n)
most_freq_test_bi = get_n_most_freq(test_bigrams, n)

print(f"There are {len(train_bigrams)} unique bigrams in the train corpus. The {n} most frequent are {str(most_freq_train_bi)}")
print(f"There are {len(test_bigrams)} unique bigrams in the train corpus. The {n} most frequent are {str(most_freq_test_bi)}\n")
    

There are 481575 unique bigrams in the train corpus. The 20 most frequent are [('. <eos>', 47153), ('<bos> Die', 4789), (', die', 4052), ('<bos> Der', 2761), ('in der', 2482), ('<bos> Das', 2014), ('<bos> "', 1895), (', der', 1883), ('" ,', 1807), (', dass', 1622), (', daß', 1380), ('in den', 1364), ('für die', 1339), ('<bos> In', 1291), ('? <eos>', 1267), ('" <eos>', 1134), ('. "', 1133), ('werden .', 1087), (', das', 1004), ('<bos> Und', 968)]
There are 484359 unique bigrams in the train corpus. The 20 most frequent are [('. <eos>', 47095), ('<bos> Die', 5011), (', die', 3900), ('<bos> Der', 2749), ('in der', 2470), ('<bos> Das', 1957), (', der', 1879), ('" ,', 1858), ('<bos> "', 1850), (', dass', 1599), ('<bos> In', 1376), ('in den', 1362), (', daß', 1346), ('für die', 1346), ('? <eos>', 1240), ('" <eos>', 1174), ('. "', 1159), ('werden .', 1077), (', das', 1025), ('" .', 918)]



### d) Compute the percentage of bigrams in the test data that have not been seen in the training data.


In [5]:
train, test = get_sets()

train_bigrams = get_n_grams(2, train)
test_bigrams = get_n_grams(2, test)

unique_keys_in_test = get_unique_keys(test_bigrams, train_bigrams)


print(f"Unique token in test: {unique_keys_in_test}. Token in test: {len(test_bigrams)}. Percentage: {round(100* unique_keys_in_test / len(test_bigrams), 2)}%")

Unique token in test: 377808. Token in test: 484359. Percentage: 78.0%


Consindering all bigrams in corpus, not only unique the numbers are as follows:  
Number of bigrams in test-data: 957335  
Number token in test, but not in training: 391930  

Percentage not seen in train: 0.40939691957360796

In [6]:
train, test = get_sets()

train_trigrams = get_n_grams(3, train)
test_trigrams = get_n_grams(3, test)

## e) Compute the percentage of trigrams in the test data that have not been seen in the training data.


In [7]:
most_freq_train_tri = get_n_most_freq(train_trigrams, n)
most_freq_test_tri = get_n_most_freq(test_trigrams, n)
unique_keys_in_test_tri = get_unique_keys(test_trigrams, train_trigrams)
print(f"There are {len(train_trigrams)} unique trigrams in the train corpus. The {n} most frequent are {str(most_freq_train_tri)}")
print()
print(f"There are {len(test_trigrams)} unique trigrams in the train corpus. The {n} most frequent are {str(most_freq_test_tri)}")
print()
print(f"Unique token in test: {unique_keys_in_test_tri}. Token in test: {len(test_trigrams)}. Percentage: {round(100* unique_keys_in_test_tri / len(test_trigrams), 2)}%")

There are 771937 unique trigrams in the train corpus. The 20 most frequent are [('. <eos> <eos>', 47153), ('<bos> <bos> Die', 4789), ('<bos> <bos> Der', 2761), ('<bos> <bos> Das', 2014), ('<bos> <bos> "', 1895), ('<bos> <bos> In', 1291), ('? <eos> <eos>', 1267), ('" <eos> <eos>', 1134), ('werden . <eos>', 1059), ('. " <eos>', 1005), ('<bos> <bos> Und', 968), ('<bos> <bos> Es', 879), ('" . <eos>', 868), ('<bos> <bos> Er', 858), ('<bos> <bos> Sie', 822), ('<bos> <bos> Im', 816), ('<bos> <bos> Ein', 707), ('<bos> <bos> Auch', 704), ('<bos> <bos> Nach', 658), ('<bos> <bos> Doch', 611)]

There are 775702 unique trigrams in the train corpus. The 20 most frequent are [('. <eos> <eos>', 47095), ('<bos> <bos> Die', 5011), ('<bos> <bos> Der', 2749), ('<bos> <bos> Das', 1957), ('<bos> <bos> "', 1850), ('<bos> <bos> In', 1376), ('? <eos> <eos>', 1240), ('" <eos> <eos>', 1174), ('werden . <eos>', 1045), ('. " <eos>', 1039), ('" . <eos>', 905), ('<bos> <bos> Und', 872), ('<bos> <bos> Es', 863), ('<b

Again, counting all trigrams, the numbers change:  
Number of trigrams in train-data: 1011850  
Number of trigrams in test-data: 1007335  
Number token in test, but not in training: 715001  

Percentage not seen in train: 0.7097946561967965

### f) How many sentences in the test data are estimated to have zero probability by an MLE bigram model from the training data?


In [9]:
def build_n_minus_one_gram(ngram):
    # split token at " ", remove last and concat
    token = ngram.split(" ")
    token = token[:-1]
    return " ".join(token)

def create_mle_model(corpus, n=2):
    n_gram_counts = get_n_grams(n, corpus)
    mle ={}
    if n ==1:
        total_token = sum(n_gram_counts.values())
        for token in n_gram_counts:
            mle[token] = n_gram_counts[token] / total_token
        return mle
    n_minus_one_gram_count = get_n_grams(n-1, corpus)
    
    if n==2:
        # for unigrams, add padding symbolds
        n_minus_one_gram_count["<bos>"] = len(corpus)
        n_minus_one_gram_count["<eos>"] = len(corpus)
    if n==3:
        n_minus_one_gram_count["<bos> <bos>"] = len(corpus)
        n_minus_one_gram_count["<eos> <eos>"] = len(corpus)
    
    
    for ngram in n_gram_counts.keys():
        c_of_w = n_gram_counts[ngram]
        
        n_minus_one_gram = build_n_minus_one_gram(ngram)
        c_of_w_minus_one = n_minus_one_gram_count[n_minus_one_gram]
        mle[ngram] = c_of_w / c_of_w_minus_one
    return mle

In [10]:
# calc mle for test corpus
def get_bigrams(document):
    bigrams = []
    tokens = document.split(" ")
    # add padding bigrams
    bigrams.append("<bos> " + tokens[0])
    bigrams.append(tokens[-1] + " <eos>")
    if len(tokens)==1:
        return bigrams
    for i in range(1, len(tokens)):
        bigrams.append(tokens[i-1] + " " + tokens[i])
    return bigrams

def get_ngram_proba(ngrams, model):
    assert type(ngrams) == list
    assert type(model) == dict
    try:
        proba = model[ngrams[0]]
        for ngram in ngrams[1:]:
            proba = proba * model[ngram]
        return proba
    except KeyError:
        return 0


def eval_mle_model(train, test):
    model = create_mle_model(train, 2)
    counter, zero_probas = 0,0
    for doc in test:
        bigrams = get_bigrams(doc)
        proba = get_ngram_proba(bigrams, model)
        counter +=1
        if proba == 0:
            zero_probas +=1
    return counter, zero_probas

train, test = get_sets()
docs, zeros = eval_mle_model(train, test)
print(f"{zeros} of {docs} documents have 0 proba (or {round(100*zeros/docs, 2)}%)")

49710 of 50000 documents have 0 proba (or 99.42%)


### g) Give the probabilities of the first 3 sentences from the test data, using a linear combination of 0-gram, unigram, bigram and trigram model with λ0 = 1.0×10−10, λ1 = 0.01, λ2 = 0.2, λ3 = 1−(λ0+λ1+λ2)

In [13]:
def transform_sent_to_ngram(sentence, n):
    for variablerCounter in range(n-1):
        sentence = "<bos> " + sentence
        sentence = sentence + " <eos>"
    tokens = sentence.split(" ")
    if n==1:
        return tokens
    n_grams =[]
    for i in range(n-1, len(tokens)):
        n_gram = tokens[i]
        for j in range(1, n):
            #append left
            n_gram = tokens[i-j] + " " + n_gram
        n_grams.append(n_gram)
    return n_grams
        
def combine_linear(val0, val1, val2,val3):
    print(val0, val1, val2, val3)
    d0 = 1.0*10**-10
    d1 = 0.01
    d2 = 0.2
    d3 = 1-d0-d1-d2
    return d0*val0 + d1 * val1 + d2*val2 + d3*val3

#wtf is 0 - gram? equal proba for each? Letters?
def get_zero_gram_proba(doc, corp):
    # TODO insert somethin useful here maybe, I just use even proba for each sentence in test set, otherwise 0
    if doc in corp:
        return 1 / len(corp)
    return 0

def lin_combo_model():
    train, test = get_sets()
    # get the mle models trained with train
    unigram_model = create_mle_model(train,1)
    bigram_model = create_mle_model(train,2)
    trigram_model = create_mle_model(train,3)

    #just take first three examples
    test = test[:3]
    probas = []
    for doc in test:
        unis = transform_sent_to_ngram(doc, 1)
        bi = transform_sent_to_ngram(doc, 2)
        tri = transform_sent_to_ngram(doc, 3)
        
        
        zeroprob = get_zero_gram_proba(doc, train)
        if zeroprob != 0:
            print(f"Zero prob is {zeroprob} for: {doc}")
        uniprob = get_ngram_proba(unis, unigram_model)
        if uniprob != 0:
            print(f"uni prob is {uniprob} for: {doc}")
        biprob = get_ngram_proba(bi, bigram_model)
        if biprob != 0:
            print(f"Bi prob is {biprob} for: {doc}")
        triprob = get_ngram_proba(tri, trigram_model)
        if zeroprob != 0:
            print(f"tri prob is {triprob} for: {doc}")
        
        res = combine_linear(zeroprob, uniprob, biprob, triprob)
        
        probas.append((doc, res))
    return probas

result = lin_combo_model()


0 0 0 0
uni prob is 3.821599123355931e-45 for: Die Preise für ein Einzelzimmer liegen hier zwischen 129 und 149 DM .
0 3.821599123355931e-45 0 0
0 0 0 0


In [14]:
print(result)

[('Stanczyk nannte es beunruhigend , daß die Bundesregierung in dieser Frage bislang nicht einmal informell Kontakt zur polnischen Regierung gesucht habe .', 0.0), ('Die Preise für ein Einzelzimmer liegen hier zwischen 129 und 149 DM .', 3.8215991233559306e-47), ('Leder : Vielleicht ringt Normann nur um Anerkennung .', 0.0)]
